# Space Time Adaptive Processing (STAP)

Radar systems on an airborne platform must often mitigate the impact of ground
clutter (i.e., radar returns from the ground or structures), which can overwhelm
other signals of interest. Airborne movement further complicates the mitigation
process because radar returns are spread in the Doppler dimension. Space-time
adaptive processing (STAP) algorithms address clutter by adaptively computing and applying filters in both the spatial (angular) and temporal (Doppler)
domains.  
\- DARPA Perfect Suite

## Radar Basics Tutorial
Excellent intro to radar processing by Michael Parker   
Covers beamforming, pulse compression (match filter), corner turns, Doppler processing, eventually gets to STAP  
https://www.eetimes.com/radar-basics-part-3-beamforming-and-radar-digital-processing/  
https://www.eetimes.com/radar-basics-part-4-space-time-adaptive-processing/  

## DARPA Perfect Suite
https://gitlab.pnnl.gov/perf-lab/perfect/perfect-suite/-/raw/master/doc/PERFECT-Suite-manual.pdf


### Data Cube
The input to STAP is a three-dimensional radar data cube consisting of L channels (or phase centers), P pulses, and N samples per pulse

N samples per pulse often called "range bins" because they sample at range intervals

Range bin dimension is called fast time. Pulse dimension is slow time

EETimes article uses:  
N pulses  
L samples per pulse  
M channels  

### Doppler Processing
First step is to convert from pulse time into Doppler spectrum domain.
Apply FFT across Pulse dimension for each range and channel.  
This gives the doppler domain (k). 

### Space-Doppler Snapshots
A snapshot is denoted as S(n,k)
Each snapshot vector is a slice of the Doppler proccessed data cube.
A snapshot consists of one range cell and one Doppler bin for all channels.

Usually neighboring snapshots are analyzed as well. 
The number of snapshots used is the __Temporal Degrees of Freedom__ ($T_{DOF}$).
For a $T_{DOF}$ of 3, a data volume of three neighboring range cells and three neighboring Doppler bins would be included for all channels. The total snapshot vector would be formed by appending each of the three vectors to form a column vector of length L*3. 

### Steering Vectors

$v(d)$

Used to zoom in on a suspected target

target steering vector is formed by the cross product of the vector representing the Doppler frequency and the vector representing the antenna angle of elevation and azimuth

Steering vector has the same dimensionality as a Space-Doppler Snapshot (including $T_{DOF}$)

$D$ total steering vectors are used to represent the different steering directions.

### Covariance Matrix - Outer Product

The covariance matrix relates ____ to _____. 

It is computed by taking the covariance of a Doppler Snapshot vector, and then averaging that covariance matrix across a block of range bins. 

The Doppler Snapshot Vector has length $L * T_{DOF}$. 
The Covarinace matrix created by taking the Outer Product of the Doppler Snapshot Vector and its Hermitian.
The result has dimensions $(L * T_{DOF}) X (L * T_{DOF})$.

The range cell dimension N is broken into M equal blocks of length $N_{R}$. 


Exerpt from [EETimes radar basics STAP article](https://www.eetimes.com/radar-basics-part-4-space-time-adaptive-processing/):  
> 
> Shown in Figure 8 is a plot of S I -1 , the inverted covariance matrix. In this case, there is a jammer at 60 degrees azimuth angle, and a target at 45 degrees, 1723 meters range and normalized Doppler of 0.11.  
> 
> ![Figure 8. Logarithmic plot of inverted covariance matrix](./inverted_covariance_matrix.jpg)
> Figure 8. Logarithmic plot of inverted covariance matrix  
> 
> Notice the very small values, on the order of –80 dB, present at 60 degrees. The STAP filtering process is detecting the correlation associated with the jammer location at 60 degrees. But inverting the covariance matrix, this jammer will be severly attenuated. Notice also the diagonal yellow clutter line. This is a side looking airborne radar, so the ground clutter has positive Doppler looking in the forward direction, or angle and negative Doppler in the backward direction or angle. This ground clutter is being attenuated at about –30 dB, proportionally less severely than the more powerful jammer signal.
> 
> The target is not present in this plot. Recall that the estimated covariance matrix is determined in range bins surrounding, but not at the expected range of the target. 

### Weighting Vectors - Inner Product, linear system solve

Weighting vectors are computed to cancel out clutter. 
They're applied to snapshot vectors. 

To generate the weighting vectors, the inverse of the Covariance Matrix is multiplied by each steering vector. 

$w_{opt} = γ R^{-1} v$

$w_{opt} = γ R^{-1}(b,k) v(d)$

$w$ - weighting vector  
$γ$ - a scalar constant  
$R$ - covaraince matrix  
$v$ - steering vector  

A weighting vector is computed for each direction $v(d)$, range bin block $N_{R}$, and Doppler bin $k$.

There are different approaches to solving for $w$. 
Perfect Suite uses QR dcomposition to solve the linear system:

$ R(b,k) w(b,k,d) = v(d)$

Choleski Decomposition is another common approach.


